###### code that calculates performance on the stop-and-ask schema prediction questions during video viewing on day 2

In [ ]:
import pandas as pd
import os
import numpy as np
import ast
import fnmatch
import glob
import matplotlib.pyplot as plt

In [ ]:
path = '/Volumes/norman/scollin/schema/data/behav/sub*'

files = [f for f in glob.glob(path + "**/*_viewing.csv", recursive=True)]

filename_list = sorted(files)
filename_list

In [ ]:
all_data = pd.DataFrame()
for i in filename_list:
    data = pd.read_csv(i, usecols = ['northOrSouth','stimFile2','key_resp_2.keys','key_resp_3.keys'])
    all_data = pd.concat([all_data,data])
#print(all_data)

In [ ]:
def convert_line(line):
    '''convert a string to a list, then extract the last element 
    
    e.g.
        ['2', '1'] -> 1
        ['2'] -> 2
        None -> 0
        
    '''
    line_list = ast.literal_eval(line)
    if line_list is None or 'NaN': 
        return 0
    else:
        return line_list[-1]
    
    
def simplify_condition_string(condition_string1,condition_string2):
    '''condsFile -> two-letter code 
    '''
    new_name =''
    if 'south' in condition_string1:
        new_name+='S'
    elif 'north' in condition_string1:
        new_name+='N'
    else:
        raise ValueError('Neither S nor N is in the condition name')

    if '2a' in condition_string2:
        new_name+='A'
    elif '2b' in condition_string2:
        new_name+='B'
    else:
        raise ValueError('Neither A nor B is in the condition name')
    return new_name

def check_answer(schema_path, pred2, pred3):
    correct = np.array([False, False])
    if schema_path == 'SA':
        if pred2 == '2': 
            correct[0] = True
        if pred3 == '1':
            correct[1] = True
    elif schema_path == 'SB':
        if pred2 == '1':
            correct[0] = True
        if pred3 == '2':
            correct[1] = True
    elif schema_path == 'NA':
        if pred2 == '1':
            correct[0] = True                    
        if pred3 == '1':        
            correct[1] = True        
    elif schema_path == 'NB':
        if pred2 == '2': 
            correct[0] = True                    
        if pred3 == '2':                
            correct[1] = True                    
    return correct

In [ ]:
BlockStarts = all_data.loc[[5]]
BlockEnds = all_data.loc[[60]]

In [ ]:
BlockStarts.replace(to_replace=['None'], value=np.nan, inplace=True)
BlockEnds.replace(to_replace=['None'], value=np.nan, inplace=True)

In [ ]:
BlockStarts = BlockStarts.fillna("['0']")
BlockEnds = BlockEnds.fillna("['0']")

In [ ]:
namesSt,key2St,key3St =[],[],[]
for i, row in BlockStarts.iterrows():
    # condition name 
    long_condition_string1 = row['northOrSouth']
    long_condition_string2 = row['stimFile2']
    short_condition_string = simplify_condition_string(long_condition_string1,long_condition_string2)
    namesSt.append(short_condition_string)    
    # prediction 2 
    prediction2 = row['key_resp_2.keys']
    
    if len(prediction2) > 1:
        prediction2 = ast.literal_eval(prediction2)
        prediction2 = prediction2[-1]
    else:
        prediction2 = ast.literal_eval(prediction2)
    
    key2St.append(prediction2)
    # prediction 3 
    prediction3 = row['key_resp_3.keys']
    
    if len(prediction3) > 1:
        prediction3 = ast.literal_eval(prediction3)
        prediction3 = prediction3[-1]
    else:
        prediction3 = ast.literal_eval(prediction3)
    
    key3St.append(prediction3)    
    
#print(namesSt)
#print(key2St)
#print(key3St)

In [ ]:
# build df 

dSt = {
    'SchemaPath':namesSt,
    'Prediction2':key2St,
    'Prediction3':key3St
}

dfSt=pd.DataFrame(data=dSt)


# check "corrects"

dfSt_correct = pd.DataFrame(columns=dSt.keys(), dtype=object)
for i, row in dfSt.iterrows():
    correct = check_answer(row['SchemaPath'], row['Prediction2'], row['Prediction3'])
    dfSt_correct.loc[i] = [row['SchemaPath'], int(correct[0]), int(correct[1])]

In [ ]:
dfSt_correct.to_csv(r'/Volumes/norman/scollin/schema/data/behav/results/predictionsResults_StartWedd_Day2.csv')


# final wedd of each block

In [ ]:
namesEnd,key2End,key3End =[],[],[]
for i, row in BlockEnds.iterrows():
    long_condition_string1 = row['northOrSouth']
    long_condition_string2 = row['stimFile2']
    short_condition_string = simplify_condition_string(long_condition_string1,long_condition_string2)
    namesEnd.append(short_condition_string)    
    # prediction 2 
    prediction2 = row['key_resp_2.keys']
    
    if len(prediction2) > 1:
        prediction2 = ast.literal_eval(prediction2)
        prediction2 = prediction2[-1]
    else:
        prediction2 = ast.literal_eval(prediction2)
    
    key2End.append(prediction2)
    # prediction 3 
    prediction3 = row['key_resp_3.keys']
    
    if len(prediction3) > 1:
        prediction3 = ast.literal_eval(prediction3)
        prediction3 = prediction3[-1]
    else:
        prediction3 = ast.literal_eval(prediction3)
    
    key3End.append(prediction3)    
    
#print(namesEnd)
#print(key2End)
#print(key3End)

In [ ]:
# build df 

dEnd = {
    'SchemaPath':namesEnd,
    'Prediction2':key2End,
    'Prediction3':key3End
}

dfEnd=pd.DataFrame(data=dEnd)


# check "corrects"

dfEnd_correct = pd.DataFrame(columns=dEnd.keys(), dtype=object)
for i, row in dfEnd.iterrows():
    correct = check_answer(row['SchemaPath'], row['Prediction2'], row['Prediction3'])
    dfEnd_correct.loc[i] = [row['SchemaPath'], int(correct[0]), int(correct[1])]

In [ ]:
dfEnd_correct.to_csv(r'/Volumes/norman/scollin/schema/data/behav/results/predictionsResults_EndWedd_Day2.csv')

In [ ]:
ResultsTotalMatrix = pd.concat((dfSt_correct,dfEnd_correct), axis=1)

In [ ]:
# remove schema path index
del ResultsTotalMatrix['SchemaPath']

In [ ]:
ResultsTotalMatrix['mean'] = ResultsTotalMatrix.mean(axis=1)

In [ ]:
ResultsTotalMatrix = ResultsTotalMatrix.astype(float)

save the mean for each participant

In [ ]:
ResultsTotalMatrix = ResultsTotalMatrix.drop('Prediction2', axis=1)
ResultsTotalMatrix = ResultsTotalMatrix.drop('Prediction3', axis=1)

In [ ]:
ResultsTotalMatrix.to_csv(r'/Volumes/norman/scollin/schema/data/behav/results/SchemaPrediction_day2.csv')
